## Target prioritisation validation based on Open Targets genetic score for IBD

### Export Target-IBD associations from Open Targets platform
##### Sources of evidence: Genetic evidence (excluding PanelApp, gene2phenotype, Orphanet, Clingen), Animal models, Expression Atlas

### Export dataset with known IBD drug targets

### Calculate % of drug targets in first x targets from ranked list